<a href="https://colab.research.google.com/github/charan-d55/SQL-Code-Summarization/blob/main/Text_Summarization_Using_Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**1: Install All the Required Packages**

In [1]:
!nvidia-smi

Sun Feb 25 09:09:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.1 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 16.0 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 12.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 64.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 38.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 8.8 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-33qacpgi/overlay/local/bin
    changing mode of /tmp/pip-build-env-33qacpgi/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-33qacpgi/overlay/

In [2]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

#**2: Import All the Required Libraries**

In [3]:
from huggingface_hub import hf_hub_download

In [4]:
from llama_cpp import Llama

#**3: Download the Model**

In [5]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [6]:
model_path

'/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin'

#**4: Loading the Model**

In [7]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [8]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

#**5: Create a Prompt Template**

In [57]:
sql_query = """SELECT School/Club Team FROM table WHERE Years in Toronto = 2003-06"""
prompt = f"Convert the following sql query into comprehensible text:\n\n{sql_query}\n\nText:"
prompt_template=f'''{prompt}'''

#**6: Generating the Response**

In [58]:
response=lcpp_llm(prompt=prompt, max_tokens=256, temperature=0.8, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=False)

Llama.generate: prefix-match hit


In [52]:
print(response)

{'id': 'cmpl-6947a860-2148-4402-a349-f7373812a30c', 'object': 'text_completion', 'created': 1708887905, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': '\nSchool/Club Team\nYears in Toronto\n\nI\'m trying to retrieve all the rows from the table where the value of "Years in Toronto" is between 2003 and 2006, inclusive. I know this can be done using a BETWEEN operator, but I don\'t know how to write the question. Please help me!', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 42, 'completion_tokens': 83, 'total_tokens': 125}}


In [59]:
print(response["choices"][0]["text"])



We want to select all the school or club teams that played their games in Toronto during the years 2003, 2004, 2005 and 2006.

Please note that 'Years' is a column name and not a date field.


#Importing datasets


In [15]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00


In [73]:
from datasets import load_dataset

dataset = load_dataset("wikisql")

Converting dataset into data frame.

In [74]:
import pandas as pd
test_df = pd.DataFrame(dataset['test'])
test_df.head()

,phase,question,table,sql
0,1,What is terrence ross' nationality,"{'header': ['Player', 'No.', 'Nationality', 'P...",{'human_readable': 'SELECT Nationality FROM ta...
1,1,What clu was in toronto 1995-96,"{'header': ['Player', 'No.', 'Nationality', 'P...",{'human_readable': 'SELECT School/Club Team FR...
2,1,which club was in toronto 2003-06,"{'header': ['Player', 'No.', 'Nationality', 'P...",{'human_readable': 'SELECT School/Club Team FR...
3,1,how many schools or teams had jalen rose,"{'header': ['Player', 'No.', 'Nationality', 'P...",{'human_readable': 'SELECT COUNT School/Club T...
4,1,Where was Assen held?,"{'header': ['No', 'Date', 'Round', 'Circuit', ...",{'human_readable': 'SELECT Round FROM table WH...


Removing the not required Columns

In [76]:
test_df.drop(columns=['table','phase'], inplace=True)
test_df.head()

,question,sql
0,What is terrence ross' nationality,SELECT Nationality FROM table WHERE Player = T...
1,What clu was in toronto 1995-96,SELECT School/Club Team FROM table WHERE Years...
2,which club was in toronto 2003-06,SELECT School/Club Team FROM table WHERE Years...
3,how many schools or teams had jalen rose,SELECT COUNT School/Club Team FROM table WHERE...
4,Where was Assen held?,SELECT Round FROM table WHERE Circuit = Assen


Cleaning the sql column with regex

In [77]:
import re
pattern = r"{'human_readable': '(.*?)', 'sel': (\d+), 'agg': (\d+), 'conds': {'column_index': \[(.*?)\], 'operator_index': \[(.*?)\], 'condition': \['(.*?)'\]}}"

for index, row in test_df.iterrows():
    # Access data in the 'column_name' column for the current row
    column_data = row['sql']
    match = re.match(pattern, str(column_data))

    if match:
        sql_query = match.group(1)
        test_df.loc[index, 'sql'] =str(sql_query)

test_df.head()

,question,sql
0,What is terrence ross' nationality,SELECT Nationality FROM table WHERE Player = T...
1,What clu was in toronto 1995-96,SELECT School/Club Team FROM table WHERE Years...
2,which club was in toronto 2003-06,SELECT School/Club Team FROM table WHERE Years...
3,how many schools or teams had jalen rose,SELECT COUNT School/Club Team FROM table WHERE...
4,Where was Assen held?,SELECT Round FROM table WHERE Circuit = Assen


In [78]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15878 entries, 0 to 15877
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  15878 non-null  object
 1   sql       15878 non-null  object
dtypes: object(2)
memory usage: 248.2+ KB


Creating new column in the data frame named model_generated to store the model generated reponses.
Generated responses for first 50 rows.

In [88]:
# Create an empty column 'model_generated' in test_df to store the generated summaries
test_df['model_generated'] = ""

# Define a function to generate summaries and populate the 'model_generated' column
def generate_and_store_summary(row):
     data= row['sql']
     prompt = f"Convert the following sql query into comprehensible text question in one line:\n\n{data}\n\nText:"
     response=lcpp_llm(prompt=prompt, max_tokens=256, temperature=0.6, top_p=0.95, repeat_penalty=1.2, top_k=150, echo=False)

     a=response["choices"][0]["text"]

     return a

# Apply the generate_and_store_summary function to generate summaries for the first 25 articles
test_df.loc[:50, 'model_generated'] = test_df.loc[:50].apply(generate_and_store_summary, axis=1)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.gene

In [90]:
test_df.head(50)

,question,sql,model_generated
0,What is terrence ross' nationality,SELECT Nationality FROM table WHERE Player = T...,"""What is Terrence Ross's nationality?"""
1,What clu was in toronto 1995-96,SELECT School/Club Team FROM table WHERE Years...,"""What is the school or club team of the playe..."
2,which club was in toronto 2003-06,SELECT School/Club Team FROM table WHERE Years...,What is the school or club team for years 200...
3,how many schools or teams had jalen rose,SELECT COUNT School/Club Team FROM table WHERE...,How many school or club teams does Jalen Rose...
4,Where was Assen held?,SELECT Round FROM table WHERE Circuit = Assen,"""What is the round for Assen?""\n\nPlease help..."
5,What was the number of race that Kevin Curtain...,SELECT COUNT No FROM table WHERE Pole Position...,How many nos are there of pole position kevin...
6,What was the date of the race in Misano?,SELECT Date FROM table WHERE Circuit = Misano,"""What is the date for the circuit 'Misano'?""\..."
7,How many different positions did Sherbrooke Fa...,SELECT COUNT Position FROM table WHERE College...,How many positions does the Sherbrookes Fauco...
8,What are the nationalities of the player picke...,SELECT Nationality FROM table WHERE College/ju...,What is the nationality of players on the Th...
9,How many different college/junior/club teams p...,SELECT COUNT College/junior/club team FROM tab...,"How many college, junior or club teams are af..."


Calculating the ROUGE metrics Score.

In [91]:
!pip install rouge

In [92]:
from rouge import Rouge

# Initialize the ROUGE evaluator
rouge = Rouge()

# Select the first 25 rows of your DataFrame for evaluation
num_samples = 50
sampled_df = test_df.head(num_samples)

# Extract the generated summaries and reference summaries for the selected samples
generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['question'].tolist()

# Calculate ROUGE scores for the selected samples
rouge_scores = rouge.get_scores(generated_summaries, reference_summaries, avg=True)

# Print the ROUGE scores
print("ROUGE Scores:", rouge_scores)

ROUGE Scores: {'rouge-1': {'r': 0.380987855281973, 'p': 0.1767075719249933, 'f': 0.22667032869254122}, 'rouge-2': {'r': 0.13256993737256892, 'p': 0.0631666832362344, 'f': 0.08142380770449119}, 'rouge-l': {'r': 0.34844746266805104, 'p': 0.15949794569301662, 'f': 0.20512948723223057}}


Clculating the Average Cosine Similarity of first 50 pairs.


In [93]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [96]:

# Function to calculate cosine similarity
def calculate_cosine_similarity(question, model_generated):
    vectorizer = CountVectorizer().fit([question, model_generated])
    vectors = vectorizer.transform([question, model_generated])
    cosine_sim = cosine_similarity(vectors)
    return cosine_sim[0, 1]

# Apply function to first 50 rows of DataFrame
test_df['cosine_similarity'] = test_df.head(50).apply(lambda row: calculate_cosine_similarity(row['question'], row['model_generated']), axis=1)

print(test_df.head(50))

                                             question  \
0                  What is terrence ross' nationality   
1                     What clu was in toronto 1995-96   
2                   which club was in toronto 2003-06   
3            how many schools or teams had jalen rose   
4                               Where was Assen held?   
5   What was the number of race that Kevin Curtain...   
6            What was the date of the race in Misano?   
7   How many different positions did Sherbrooke Fa...   
8   What are the nationalities of the player picke...   
9   How many different college/junior/club teams p...   
10  How many different nationalities do the player...   
11                What's Dorain Anneck's pick number?   
12  What is the nationality of the player from Van...   
13  What's the pick number of the player from Spri...   
14  When were the ships launched that were laid do...   
15  List the # for ships commissioned on december ...   
16  List the # for ships commis

In [97]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15878 entries, 0 to 15877
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   question           15878 non-null  object 
 1   sql                15878 non-null  object 
 2   model_generated    15878 non-null  object 
 3   cosine_similarity  50 non-null     float64
dtypes: float64(1), object(3)
memory usage: 496.3+ KB


In [99]:
# Calculate average cosine similarity
average_cosine_similarity = test_df['cosine_similarity'].mean()

print("Average Cosine Similarity:", average_cosine_similarity)

Average Cosine Similarity: 0.4152766426615446
